## Building Simple XGBoost Bayesian Hyperparam Optimization

In [1]:
import numpy as np
import sklearn
import xgboost
import hyperopt
import pandas as pd
import time

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
from hyperopt.pyll.stochastic import sample
from sklearn import datasets, pipeline

In [2]:
#Generating a multilabel classifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
X1, y1 = make_classification(n_features=50, n_samples=1000)
X2, y2 = make_classification(n_features=50, n_samples=1000)
X = np.concatenate((X1, X2), axis=1)
y = np.transpose([y2, y1])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [3]:
#Making sure I can use cross validation score with multilabel
from xgboost import XGBClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_score
myXGBC = OneVsRestClassifier(XGBClassifier(max_depth=3, subsample=0.5), n_jobs=2)
mean_score = cross_val_score(myXGBC, X, y, cv=5, scoring='roc_auc_ovr_weighted').mean()
mean_score

0.9923685241428956

In [4]:
#Define parameter space for hyperopt
xgbc_param_hyperopt= {
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(1)),          #1
    'max_depth': scope.int(hp.quniform('max_depth', 5, 15, 1)),                        #3
    'max_delta_step': scope.int(hp.quniform('max_delta_step', 0, 10, 1)),              #2
    'booster': hp.choice('booster', ['gbtree', 'dart']),                               #0
    'subsample': hp.uniform('subsample', 0.0, 1.0),                                    #7
    'scale_pos_weight': hp.uniform('scale_pos_weight', 1, 150),                        #6
    'tree_method': hp.choice('tree_method', ['auto', 'hist']),                         #8
    'sampling_method': hp.choice('sampling_method', ['uniform', 'gradient_based']),    #5
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0)                                   #4
}

In [5]:
#Defining the hyperopt function
def hyperopt(param_space, X_train, y_train, X_test, y_test, num_eval):
    
    start = time.time()
    
    def objective_function(params):
        clf = OneVsRestClassifier(XGBClassifier(**params), n_jobs=2)
        score = cross_val_score(clf, X_train, y_train, cv=5, scoring='roc_auc_ovr_weighted').mean()
        return {'loss': -score, 'status': STATUS_OK}

    trials = Trials()
    best_param = fmin(objective_function, 
                      param_space, 
                      algo=tpe.suggest, 
                      max_evals=num_eval, 
                      trials=trials,
                      rstate= np.random.RandomState(23))
    loss = [x['result']['loss'] for x in trials.trials]
    
    best_param_values = [x for x in best_param.values()]
    
    if best_param_values[0] == 0:
        booster = 'gbtree'
    else:
        booster = 'dart'
        
        
    if best_param_values[8] == 0:
        tree_method = 'auto'
    else:
        tree_method = 'hist'
        
    
    if best_param_values[5] == 0:
        sampling_method = 'uniform'
    else:
        sampling_method = 'gradient_based'
    
    clf_best = OneVsRestClassifier(XGBClassifier(learning_rate=best_param_values[1],
                                  max_depth=int(best_param_values[3]),
                                  max_delta_step=int(best_param_values[2]),
                                  booster=booster,
                                  subsample=best_param_values[7],
                                  scale_pos_weight = best_param_values[6],
                                  tree_method = tree_method,
                                  sampling_method = sampling_method,
                                  reg_lambda = best_param_values[4],
                                    
                                                 
                                ), n_jobs=2)
                                  
    clf_best.fit(X_train, y_train)
    
    print("")
    print("##### Results")
    print("Score best parameters: ", min(loss)*-1)
    print("Best parameters: ", best_param)
    print("Test Score: ", clf_best.score(X_test, y_test))
    print("Time elapsed: ", time.time() - start)
    print("Parameter combinations evaluated: ", num_eval)
    
    return trials


In [6]:
results_hyperopt = hyperopt(xgbc_param_hyperopt, X_train, y_train, X_test, y_test, num_eval=30)

100%|██████████| 30/30 [03:58<00:00,  7.95s/trial, best loss: -0.9922728076435889]

##### Results
Score best parameters:  0.9922728076435889
Best parameters:  {'booster': 1, 'learning_rate': 0.06393618887304636, 'max_delta_step': 3.0, 'max_depth': 10.0, 'reg_lambda': 0.8009949659818361, 'sampling_method': 0, 'scale_pos_weight': 1.0618494241443832, 'subsample': 0.4605314441128805, 'tree_method': 0}
Test Score:  0.935
Time elapsed:  239.73936104774475
Parameter combinations evaluated:  30
